In [10]:

import os
import zipfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.io import read_image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from torch.optim import Adam
from glob import glob
from PIL import Image


In [11]:
# Step 1: Unzip the data files
train_zip_path = '/content/Train-20241109T060353Z-002.zip'
test_zip_path = '/content/Test-20241109T060349Z-002.zip'
data_dir = '/content/unzipped_data'

train_dir = os.path.join(data_dir, 'Train')
test_dir = os.path.join(data_dir, 'Test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(train_dir)

with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(test_dir)

In [12]:
# Step 2: Load CSV files
train_labels = pd.read_csv('/content/train (2).csv')
test_labels = pd.read_csv('/content/test (1).csv')

In [13]:
# Step 3: Map paths to image files
train_paths = glob(os.path.join(train_dir, '**', '*.jpg'), recursive=True)
test_paths = glob(os.path.join(test_dir, '**', '*.jpg'), recursive=True)

train_path_dict = {os.path.basename(path): path for path in train_paths}
test_path_dict = {os.path.basename(path): path for path in test_paths}

train_labels['path'] = train_labels['path'].map(train_path_dict)
test_labels['path'] = test_labels['path'].map(test_path_dict)

# Drop any rows where the path could not be found
train_labels.dropna(subset=['path'], inplace=True)
test_labels.dropna(subset=['path'], inplace=True)

# Print unique labels in the training set for verification
print("Unique labels in training set:", train_labels['label'].unique())

# Map labels to a continuous range if needed
label_mapping = {old_label: new_label for new_label, old_label in enumerate(sorted(train_labels['label'].unique()))}
train_labels['label'] = train_labels['label'].map(label_mapping)

# Update num_classes to reflect the remapped labels
num_classes = train_labels['label'].nunique()
print("Updated number of classes:", num_classes)

# Verify that labels are within the correct range
assert train_labels['label'].max() == num_classes - 1, "Labels are out of range for the number of classes."


Unique labels in training set: [22 18 40 25  5 14  1 16 30  9  7 12 19 24 33 26 15  0 38 34  4 29 13 43
 28 36 27 37  3 41  8 17  2 21 10 20 23 31 32  6 42 35 39]
Updated number of classes: 43


In [14]:
# Step 4: Dataset class
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, is_test=False):
        self.dataframe = dataframe
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['path']
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        if not self.is_test:
            label = self.dataframe.iloc[idx]['label']
            return image, label
        else:
            return image, img_path


In [15]:
# Step 5: Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [16]:
# Step 6: Datasets and DataLoaders
train_dataset = ImageDataset(train_labels, transform=transform)
test_dataset = ImageDataset(test_labels, transform=transform, is_test=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [17]:
# Step 7: Model definition
class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetClassifier, self).__init__()
        self.base_model = models.efficientnet_b0(pretrained=True)
        self.base_model.classifier = nn.Sequential(
            nn.Linear(self.base_model.classifier[1].in_features, num_classes)
        )

    def forward(self, x):
        return self.base_model(x)


In [18]:
# Step 8: Model initialization
num_classes = train_labels['label'].nunique()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNetClassifier(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
# Step 9: Training function
def train_model(model, dataloader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(dataloader):
            images = images.to(device)
            labels = labels.to(device).long()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

In [20]:
# Step 10: Train the model
train_model(model, train_loader, criterion, optimizer, epochs=5)

100%|██████████| 74/74 [00:41<00:00,  1.77it/s]


Epoch [1/5], Loss: 1.6055


100%|██████████| 74/74 [00:40<00:00,  1.82it/s]


Epoch [2/5], Loss: 0.7379


100%|██████████| 74/74 [00:40<00:00,  1.82it/s]


Epoch [3/5], Loss: 0.4308


100%|██████████| 74/74 [00:40<00:00,  1.82it/s]


Epoch [4/5], Loss: 0.2776


100%|██████████| 74/74 [00:40<00:00,  1.82it/s]

Epoch [5/5], Loss: 0.1852


In [21]:
# Step 11: Save the original model to Google Drive
drive_model_path = '/content/drive/MyDrive/saved_models/efficientnet_model.pth'
os.makedirs(os.path.dirname(drive_model_path), exist_ok=True)
torch.save(model.state_dict(), drive_model_path)
print(f"Original model saved to {drive_model_path}")

Original model saved to /content/drive/MyDrive/saved_models/efficientnet_model.pth


In [22]:
# Step 12: Convert to TorchScript and save
model_scripted = torch.jit.script(model)
model_scripted = model_scripted.to(device)
drive_scripted_path = '/content/drive/MyDrive/saved_models/efficientnet_model_scripted.pth'
torch.jit.save(model_scripted, drive_scripted_path)
print(f"Scripted model saved to {drive_scripted_path}")

Scripted model saved to /content/drive/MyDrive/saved_models/efficientnet_model_scripted.pth


In [23]:
# Step 13: Model evaluation for training (optional step)
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device).long()
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [24]:
# Step 14: Calculate F1 score and accuracy
f1 = f1_score(all_labels, all_preds, average='weighted')
accuracy = accuracy_score(all_labels, all_preds)
print(f"Training Accuracy: {accuracy:.4f}")
print(f"Training F1 Score (weighted): {f1:.4f}")

model_scripted = torch.jit.script(model)
model_scripted = model_scripted.to(device)
print("Model converted to TorchScript for optimized inference.")

Training Accuracy: 0.9822
Training F1 Score (weighted): 0.9817
Model converted to TorchScript for optimized inference.


In [25]:
# Step 15: Make predictions on the test set using the optimized model
test_preds = []
test_paths = []

model_scripted.eval()
with torch.no_grad():
    for images, paths in test_loader:
        images = images.to(device)
        outputs = model_scripted(images)
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy())
        test_paths.extend(paths)

# Create DataFrame for submission
test_results = pd.DataFrame({'path': test_paths, 'label': test_preds})
test_results['path'] = test_results['path'].apply(os.path.basename)

# Save to CSV
test_results.to_csv('/content/test_predictions.csv', index=False)
print("Test predictions saved to test_predictions.csv")


Test predictions saved to test_predictions.csv
